In [61]:
# base_path = './'

In [62]:
import pandas as pd
from time import time
import tensorflow as tf
import os

In [63]:
def gerar_rede(num_classes: int, n_neurons: int, n_hidden_layers: int):
    dir_path = os.getcwd()
    
    metrics_file = f"{dir_path}/training_metrics.csv"
    if os.path.exists(metrics_file):
        metrics_df = pd.read_csv(metrics_file)
    else:
        metrics_df = pd.DataFrame(columns=["Model Name", "Training Time", "Train Accuracy", "Test Accuracy"])

    file_data_train = os.path.join(dir_path, "train.csv")
    file_data_test = os.path.join(dir_path, "test.csv")
    
    data_train = pd.read_csv(file_data_train).to_numpy()
    data_test = pd.read_csv(file_data_test).to_numpy()

    x_train, y_train = data_train[:, :-1], data_train[:, -1]
    x_test, y_test = data_test[:, :-1], data_test[:, -1]

    y_train_ohe = tf.keras.utils.to_categorical(
        y_train, num_classes=num_classes)
    y_test_ohe = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)

    model = tf.keras.Sequential(
        [
            tf.keras.layers.Input(shape=[x_train.shape[1]]),
        ]
    )

    for _ in range(n_hidden_layers):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))

    model.add(tf.keras.layers.Dense(num_classes, activation="softmax"))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    
    extentions = ['h5', 'keras']
    model_path = os.path.join(
        dir_path, "models", f"model_{n_hidden_layers}layers_{n_neurons}neurons"
    )
    if not os.path.join(dir_path, "models"):
        os.makedirs(os.path.join(dir_path, "models"))
    

    es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=100)
    ck = tf.keras.callbacks.ModelCheckpoint(
        f'{model_path}.{extentions[1]}', monitor="val_accuracy", save_best_only=True
    )

    start = time()
    model.fit(
        x_train,
        y_train_ohe,
        batch_size=4,
        epochs=100,
        validation_data=(x_test, y_test_ohe),
        verbose=2,
        callbacks=[ck, es],
    )
    end = time() 
    
    print(f"Tempo de Treinamento: {end-start}")

    # salvar modelo
    model = tf.keras.models.load_model(f'{model_path}.{extentions[1]}')

    # avaliar modelo com os dados de treinamento
    print("Resultado Treinamento")
    model.evaluate(x_train, y_train_ohe, verbose=2)

    # avaliar modelo com os dados de teste
    print("Resultado Teste")
    # model.evaluate(x_test, y_test_ohe, verbose=2)
    model.save(f'{model_path}.{extentions[0]}')
    
    train_loss, train_acc = model.evaluate(x_train, y_train_ohe, verbose=2)
    test_loss, test_acc = model.evaluate(x_test, y_test_ohe, verbose=2)
    
    metrics_df.loc[len(metrics_df)] = { # type: ignore
        "Model Name": f"model_{n_hidden_layers}layers_{n_neurons}neurons",
        "Training Time": end - start,
        "Train Accuracy": train_acc,
        "Test Accuracy": test_acc
    } 
    metrics_df.to_csv(metrics_file, index=False)

In [64]:
dataset = pd.read_csv('data.csv')
# verificaf quantas classes tem
num_classes = len(dataset['target'].value_counts())


In [65]:
for i in range(1, 4):
    n_hidden_layers = i
    n_neurons = 5
    gerar_rede(num_classes, n_neurons, n_hidden_layers)
    

Epoch 1/100


41/41 - 1s - 23ms/step - accuracy: 0.2988 - loss: 1.6169 - val_accuracy: 0.4390 - val_loss: 1.5895
Epoch 2/100
41/41 - 0s - 3ms/step - accuracy: 0.4207 - loss: 1.5760 - val_accuracy: 0.4634 - val_loss: 1.5641
Epoch 3/100
41/41 - 0s - 4ms/step - accuracy: 0.4329 - loss: 1.5427 - val_accuracy: 0.4634 - val_loss: 1.5403
Epoch 4/100
41/41 - 0s - 3ms/step - accuracy: 0.4268 - loss: 1.5102 - val_accuracy: 0.4634 - val_loss: 1.5192
Epoch 5/100
41/41 - 0s - 3ms/step - accuracy: 0.4207 - loss: 1.4801 - val_accuracy: 0.4634 - val_loss: 1.4990
Epoch 6/100
41/41 - 0s - 4ms/step - accuracy: 0.4146 - loss: 1.4521 - val_accuracy: 0.4146 - val_loss: 1.4798
Epoch 7/100
41/41 - 0s - 2ms/step - accuracy: 0.4146 - loss: 1.4254 - val_accuracy: 0.4146 - val_loss: 1.4645
Epoch 8/100
41/41 - 0s - 2ms/step - accuracy: 0.3963 - loss: 1.4008 - val_accuracy: 0.3902 - val_loss: 1.4510
Epoch 9/100
41/41 - 0s - 2ms/step - accuracy: 0.3902 - loss: 1.3790 - val_accuracy: 0.3902 - val_loss: 1.4402
Epoch 10/100
41/41 - 

Resultado Teste
6/6 - 0s - 5ms/step - accuracy: 0.6037 - loss: 0.8515
2/2 - 0s - 10ms/step - accuracy: 0.6341 - loss: 0.9774
Epoch 1/100
41/41 - 1s - 23ms/step - accuracy: 0.0488 - loss: 1.7383 - val_accuracy: 0.1463 - val_loss: 1.6071
Epoch 2/100
41/41 - 0s - 2ms/step - accuracy: 0.2073 - loss: 1.6171 - val_accuracy: 0.3659 - val_loss: 1.5605
Epoch 3/100
41/41 - 0s - 4ms/step - accuracy: 0.4573 - loss: 1.5658 - val_accuracy: 0.4146 - val_loss: 1.5487
Epoch 4/100
41/41 - 0s - 2ms/step - accuracy: 0.4207 - loss: 1.5444 - val_accuracy: 0.4146 - val_loss: 1.5330
Epoch 5/100
41/41 - 0s - 3ms/step - accuracy: 0.3780 - loss: 1.5275 - val_accuracy: 0.4146 - val_loss: 1.5147
Epoch 6/100
41/41 - 0s - 2ms/step - accuracy: 0.3720 - loss: 1.5087 - val_accuracy: 0.4146 - val_loss: 1.4941
Epoch 7/100
41/41 - 0s - 2ms/step - accuracy: 0.3598 - loss: 1.4880 - val_accuracy: 0.3902 - val_loss: 1.4719
Epoch 8/100
41/41 - 0s - 3ms/step - accuracy: 0.3598 - loss: 1.4660 - val_accuracy: 0.3902 - val_loss: 1

Resultado Teste
6/6 - 0s - 4ms/step - accuracy: 0.6829 - loss: 0.8562
2/2 - 0s - 10ms/step - accuracy: 0.6585 - loss: 0.9709
Epoch 1/100
41/41 - 1s - 23ms/step - accuracy: 0.1768 - loss: 1.6160 - val_accuracy: 0.3659 - val_loss: 1.5990
Epoch 2/100
41/41 - 0s - 5ms/step - accuracy: 0.3110 - loss: 1.5868 - val_accuracy: 0.3415 - val_loss: 1.5869
Epoch 3/100
41/41 - 0s - 2ms/step - accuracy: 0.3415 - loss: 1.5708 - val_accuracy: 0.3659 - val_loss: 1.5754
Epoch 4/100
41/41 - 0s - 2ms/step - accuracy: 0.2988 - loss: 1.5561 - val_accuracy: 0.2927 - val_loss: 1.5638
Epoch 5/100
41/41 - 0s - 2ms/step - accuracy: 0.3476 - loss: 1.5401 - val_accuracy: 0.3415 - val_loss: 1.5515
Epoch 6/100
41/41 - 0s - 4ms/step - accuracy: 0.3780 - loss: 1.5206 - val_accuracy: 0.3415 - val_loss: 1.5389
Epoch 7/100
41/41 - 0s - 2ms/step - accuracy: 0.3780 - loss: 1.4939 - val_accuracy: 0.3415 - val_loss: 1.5312
Epoch 8/100
41/41 - 0s - 2ms/step - accuracy: 0.3780 - loss: 1.4672 - val_accuracy: 0.3415 - val_loss: 1

Resultado Teste
6/6 - 0s - 3ms/step - accuracy: 0.5488 - loss: 0.9513
2/2 - 0s - 8ms/step - accuracy: 0.5610 - loss: 1.3840
